In [ ]:
import pandas as pd
import glob
import os

root_path = '/Users/liuZhesan/Desktop/Columbia University/25fall/Data Analytics/Assignment/Final_project/final_project_personal_data'
all_files = glob.glob(os.path.join(root_path, '**', '*.csv'), recursive=True)

print(f"共找到 {len(all_files)} 个 CSV 文件")
df_list = []

# 3. 逐个读取文件
for file_path in all_files:
    try:
        print("正在读取：", os.path.basename(file_path))
        df = pd.read_csv(file_path, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
        print(f"⚠️ {os.path.basename(file_path)} 用 latin1 编码读取成功")
    df_list.append(df)

# 4. 合并所有数据
merged_df = pd.concat(df_list, ignore_index=True)
print("✅ 合并完成！总行数：", len(merged_df))


In [7]:
merged_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C40E88BB937BDC15,electric_bike,2024-09-26 09:06:40.740,2024-09-26 09:27:55.009,Columbia St & Degraw St,4422.04,6 Ave & Canal St,5500.07,40.685930,-74.002424,40.722438,-74.005664,casual
1,F9DB9FB8DF1B9698,electric_bike,2024-09-15 16:36:01.749,2024-09-15 16:50:41.891,Kent Ave & Grand St,5388.01,Pike St & E Broadway,5270.05,40.716425,-73.965940,40.714067,-73.992939,casual
2,2C876366CD11D036,electric_bike,2024-09-24 22:44:19.582,2024-09-24 22:59:35.506,W 67 St & Broadway,7116.04,Central Park North & Adam Clayton Powell Blvd,7617.07,40.774925,-73.982666,40.799484,-73.955613,casual
3,D639068AC8D3DE4D,classic_bike,2024-09-16 16:39:05.447,2024-09-16 17:02:05.830,Madison Ave & E 82 St,7188.13,Central Park North & Adam Clayton Powell Blvd,7617.07,40.778131,-73.960694,40.799484,-73.955613,casual
4,7C1B06CFC9849B71,electric_bike,2024-09-30 15:45:18.183,2024-09-30 15:46:42.613,Lafayette Ave & St James Pl,4494.04,Willoughby Ave & Hall St,4611.03,40.688515,-73.964763,40.691960,-73.965369,casual


In [3]:
min_lat, max_lat = 40.799, 40.825
min_lng, max_lng = -73.976, -73.946
def in_box(lat_col, lng_col):
    return (
        (merged_df[lat_col] >= min_lat) & (merged_df[lat_col] <= max_lat) &
        (merged_df[lng_col] >= min_lng) & (merged_df[lng_col] <= max_lng)
    )

in_box_start = in_box('start_lat', 'start_lng')
in_box_end   = in_box('end_lat',   'end_lng')

df_columbia_any = merged_df[in_box_start | in_box_end].copy()

start_stations = df_columbia_any.loc[in_box_start, 
    ['start_station_id','start_station_name','start_lat','start_lng']
].rename(columns={
    'start_station_id':'station_id','start_station_name':'station_name',
    'start_lat':'lat','start_lng':'lng'
})

end_stations = df_columbia_any.loc[in_box_end, 
    ['end_station_id','end_station_name','end_lat','end_lng']
].rename(columns={
    'end_station_id':'station_id','end_station_name':'station_name',
    'end_lat':'lat','end_lng':'lng'
})

stations_in_box = (
    pd.concat([start_stations, end_stations], ignore_index=True)
      .dropna(subset=['lat','lng'])
      .drop_duplicates(subset=['station_id','station_name'])
      .sort_values('station_name')
      .reset_index(drop=True)
)


NameError: name 'merged_df' is not defined

In [10]:
# 每个站点作为起点的出行次数（范围内）
starts_count = (
    df_columbia_any[in_box_start]
    .groupby(['start_station_id','start_station_name'], dropna=False)
    .size().reset_index(name='trip_count_from_here')
    .sort_values('trip_count_from_here', ascending=False)
)

# 每个站点作为终点的出行次数（范围内）
ends_count = (
    df_columbia_any[in_box_end]
    .groupby(['end_station_id','end_station_name'], dropna=False)
    .size().reset_index(name='trip_count_to_here')
    .sort_values('trip_count_to_here', ascending=False)
)


/var/folders/mp/krm2l_md3rg7ymrqgfg4stsr0000gn/T/ipykernel_19257/2610173307.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_columbia_any[in_box_start]
/var/folders/mp/krm2l_md3rg7ymrqgfg4stsr0000gn/T/ipykernel_19257/2610173307.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_columbia_any[in_box_end]


In [12]:
starts_count,ends_count

(   start_station_id                             start_station_name  \
 50          7617.07  Central Park North & Adam Clayton Powell Blvd   
 63          7680.03                      Cathedral Pkwy & Broadway   
 55          7631.23             Frederick Douglass Blvd & W 112 St   
 64          7688.12             Frederick Douglass Blvd & W 117 St   
 58          7646.04                       W 110 St & Amsterdam Ave   
 ..              ...                                            ...   
 42          7925.04                              W 133 St & 12 Ave   
 48          7981.27                        W 140 St & Riverside Dr   
 37          7881.09                        Old Broadway & W 133 St   
 71          7727.08                       Amsterdam Ave & W 119 St   
 21          7727.08                       Amsterdam Ave & W 119 St   
 
     trip_count_from_here  
 50                 89708  
 63                 82612  
 55                 75804  
 64                 74128  
 58   

In [2]:
print(sum(starts_count))

NameError: name 'starts_count' is not defined